# Pre_processing data for training

* Khai báo thư viện

In [20]:
import pandas as pd
import numpy as np
import re
import warnings

# Cấu hình hiển thị và tắt cảnh báo không cần thiết
warnings.filterwarnings('ignore')

# Cấu hình dữ liệu & từ điển

* 1. Từ điển dữ liệu nguyên tử (Magpie Data) dùng để trích xuất đặc trưng
  - Bao gồm: Khối lượng nguyên tử, Bán kính, Độ âm điện, Hóa trị

In [21]:
ATOM_DATA = {
    'H': {'mass': 1.008, 'rad': 0.53, 'eneg': 2.20, 'val': 1}, 'He': {'mass': 4.0026, 'rad': 0.31, 'eneg': 0.0, 'val': 0},
    'Li': {'mass': 6.94, 'rad': 1.67, 'eneg': 0.98, 'val': 1}, 'Be': {'mass': 9.012, 'rad': 1.12, 'eneg': 1.57, 'val': 2},
    'B': {'mass': 10.81, 'rad': 0.87, 'eneg': 2.04, 'val': 3}, 'C': {'mass': 12.011, 'rad': 0.67, 'eneg': 2.55, 'val': 4},
    'N': {'mass': 14.007, 'rad': 0.56, 'eneg': 3.04, 'val': 5}, 'O': {'mass': 15.999, 'rad': 0.48, 'eneg': 3.44, 'val': 6},
    'F': {'mass': 18.998, 'rad': 0.42, 'eneg': 3.98, 'val': 7}, 'Ne': {'mass': 20.180, 'rad': 0.38, 'eneg': 0.0, 'val': 0},
    'Na': {'mass': 22.990, 'rad': 1.90, 'eneg': 0.93, 'val': 1}, 'Mg': {'mass': 24.305, 'rad': 1.45, 'eneg': 1.31, 'val': 2},
    'Al': {'mass': 26.982, 'rad': 1.18, 'eneg': 1.61, 'val': 3}, 'Si': {'mass': 28.085, 'rad': 1.11, 'eneg': 1.90, 'val': 4},
    'P': {'mass': 30.974, 'rad': 0.98, 'eneg': 2.19, 'val': 5}, 'S': {'mass': 32.06, 'rad': 0.88, 'eneg': 2.58, 'val': 6},
    'Cl': {'mass': 35.45, 'rad': 0.79, 'eneg': 3.16, 'val': 7}, 'Ar': {'mass': 39.948, 'rad': 0.71, 'eneg': 0.0, 'val': 0},
    'K': {'mass': 39.098, 'rad': 2.43, 'eneg': 0.82, 'val': 1}, 'Ca': {'mass': 40.078, 'rad': 1.94, 'eneg': 1.00, 'val': 2},
    'Sc': {'mass': 44.956, 'rad': 1.84, 'eneg': 1.36, 'val': 3}, 'Ti': {'mass': 47.867, 'rad': 1.76, 'eneg': 1.54, 'val': 4},
    'V': {'mass': 50.942, 'rad': 1.71, 'eneg': 1.63, 'val': 5}, 'Cr': {'mass': 51.996, 'rad': 1.66, 'eneg': 1.66, 'val': 6},
    'Mn': {'mass': 54.938, 'rad': 1.61, 'eneg': 1.55, 'val': 7}, 'Fe': {'mass': 55.845, 'rad': 1.56, 'eneg': 1.83, 'val': 8},
    'Co': {'mass': 58.933, 'rad': 1.52, 'eneg': 1.88, 'val': 9}, 'Ni': {'mass': 58.693, 'rad': 1.49, 'eneg': 1.91, 'val': 10},
    'Cu': {'mass': 63.546, 'rad': 1.45, 'eneg': 1.90, 'val': 11}, 'Zn': {'mass': 65.38, 'rad': 1.42, 'eneg': 1.65, 'val': 12},
    'Ga': {'mass': 69.723, 'rad': 1.36, 'eneg': 1.81, 'val': 3}, 'Ge': {'mass': 72.63, 'rad': 1.25, 'eneg': 2.01, 'val': 4},
    'As': {'mass': 74.922, 'rad': 1.14, 'eneg': 2.18, 'val': 5}, 'Se': {'mass': 78.96, 'rad': 1.03, 'eneg': 2.55, 'val': 6},
    'Br': {'mass': 79.904, 'rad': 0.94, 'eneg': 2.96, 'val': 7}, 'Kr': {'mass': 83.798, 'rad': 0.88, 'eneg': 3.00, 'val': 0},
    'Rb': {'mass': 85.468, 'rad': 2.65, 'eneg': 0.82, 'val': 1}, 'Sr': {'mass': 87.62, 'rad': 2.15, 'eneg': 0.95, 'val': 2},
    'Y': {'mass': 88.906, 'rad': 2.12, 'eneg': 1.22, 'val': 3}, 'Zr': {'mass': 91.224, 'rad': 2.06, 'eneg': 1.33, 'val': 4},
    'Nb': {'mass': 92.906, 'rad': 1.98, 'eneg': 1.60, 'val': 5}, 'Mo': {'mass': 95.96, 'rad': 1.90, 'eneg': 2.16, 'val': 6},
    'Tc': {'mass': 98.0, 'rad': 1.83, 'eneg': 1.90, 'val': 7}, 'Ru': {'mass': 101.07, 'rad': 1.78, 'eneg': 2.20, 'val': 8},
    'Rh': {'mass': 102.91, 'rad': 1.73, 'eneg': 2.28, 'val': 9}, 'Pd': {'mass': 106.42, 'rad': 1.69, 'eneg': 2.20, 'val': 10},
    'Ag': {'mass': 107.87, 'rad': 1.65, 'eneg': 1.93, 'val': 11}, 'Cd': {'mass': 112.41, 'rad': 1.61, 'eneg': 1.69, 'val': 12},
    'In': {'mass': 114.82, 'rad': 1.56, 'eneg': 1.78, 'val': 3}, 'Sn': {'mass': 118.71, 'rad': 1.45, 'eneg': 1.96, 'val': 4},
    'Sb': {'mass': 121.76, 'rad': 1.33, 'eneg': 2.05, 'val': 5}, 'Te': {'mass': 127.60, 'rad': 1.23, 'eneg': 2.10, 'val': 6},
    'I': {'mass': 126.90, 'rad': 1.15, 'eneg': 2.66, 'val': 7}, 'Xe': {'mass': 131.29, 'rad': 1.08, 'eneg': 2.60, 'val': 0},
    'Cs': {'mass': 132.91, 'rad': 2.98, 'eneg': 0.79, 'val': 1}, 'Ba': {'mass': 137.33, 'rad': 2.53, 'eneg': 0.89, 'val': 2},
    'La': {'mass': 138.91, 'rad': 1.95, 'eneg': 1.10, 'val': 3}, 'Ce': {'mass': 140.12, 'rad': 1.85, 'eneg': 1.12, 'val': 3},
    'Pr': {'mass': 140.91, 'rad': 2.47, 'eneg': 1.13, 'val': 3}, 'Nd': {'mass': 144.24, 'rad': 2.06, 'eneg': 1.14, 'val': 3},
    'Pm': {'mass': 145.00, 'rad': 2.05, 'eneg': 1.13, 'val': 3}, 'Sm': {'mass': 150.36, 'rad': 2.38, 'eneg': 1.17, 'val': 3},
    'Eu': {'mass': 151.96, 'rad': 2.31, 'eneg': 1.20, 'val': 2}, 'Gd': {'mass': 157.25, 'rad': 2.33, 'eneg': 1.20, 'val': 3},
    'Tb': {'mass': 158.93, 'rad': 2.25, 'eneg': 1.20, 'val': 3}, 'Dy': {'mass': 162.50, 'rad': 2.28, 'eneg': 1.22, 'val': 3},
    'Ho': {'mass': 164.93, 'rad': 2.26, 'eneg': 1.23, 'val': 3}, 'Er': {'mass': 167.26, 'rad': 2.26, 'eneg': 1.24, 'val': 3},
    'Tm': {'mass': 168.93, 'rad': 2.22, 'eneg': 1.25, 'val': 3}, 'Yb': {'mass': 173.05, 'rad': 2.22, 'eneg': 1.10, 'val': 2},
    'Lu': {'mass': 174.97, 'rad': 2.17, 'eneg': 1.27, 'val': 3}, 'Hf': {'mass': 178.49, 'rad': 2.08, 'eneg': 1.30, 'val': 4},
    'Ta': {'mass': 180.95, 'rad': 2.00, 'eneg': 1.50, 'val': 5}, 'W': {'mass': 183.84, 'rad': 1.93, 'eneg': 2.36, 'val': 6},
    'Re': {'mass': 186.21, 'rad': 1.88, 'eneg': 1.90, 'val': 7}, 'Os': {'mass': 190.23, 'rad': 1.85, 'eneg': 2.20, 'val': 8},
    'Ir': {'mass': 192.22, 'rad': 1.80, 'eneg': 2.20, 'val': 9}, 'Pt': {'mass': 195.08, 'rad': 1.77, 'eneg': 2.28, 'val': 10},
    'Au': {'mass': 196.97, 'rad': 1.74, 'eneg': 2.54, 'val': 11}, 'Hg': {'mass': 200.59, 'rad': 1.71, 'eneg': 2.00, 'val': 12},
    'Tl': {'mass': 204.38, 'rad': 1.56, 'eneg': 1.62, 'val': 3}, 'Pb': {'mass': 207.20, 'rad': 1.54, 'eneg': 2.33, 'val': 4},
    'Bi': {'mass': 208.98, 'rad': 1.43, 'eneg': 2.02, 'val': 5}, 'Po': {'mass': 209.00, 'rad': 1.35, 'eneg': 2.00, 'val': 6},
    'Th': {'mass': 232.04, 'rad': 1.80, 'eneg': 1.30, 'val': 4}, 'Pa': {'mass': 231.04, 'rad': 1.63, 'eneg': 1.50, 'val': 5},
    'U': {'mass': 238.03, 'rad': 1.56, 'eneg': 1.38, 'val': 6}
}

* 2. Quy tắc chuyên gia (Expert Rules) để sửa lỗi dữ liệu
   - Cấu trúc: 'Công thức': {'fix_pressure': GPa, 'max_tc': K, 'min_tc': K}

In [22]:
EXPERT_RULES = {
    # Nhóm Hydrides (Siêu dẫn nhiệt độ cao dưới áp suất lớn)
    'LaH10': {'fix_pressure': 170.0, 'min_tc': 200}, 
    'H3S':   {'fix_pressure': 155.0, 'min_tc': 180},
    'YH9':   {'fix_pressure': 243.0, 'min_tc': 200},
    'YH6':   {'fix_pressure': 166.0, 'min_tc': 200},
    'CaH6':  {'fix_pressure': 172.0, 'min_tc': 200},
    'ThH10': {'fix_pressure': 161.0, 'min_tc': 150},
    # Nhóm vật liệu thường (MgB2, Fe-based, Cuprates)
    'MgB2':  {'fix_pressure': 0.0, 'max_tc': 45.0}, 
    'FeSe':  {'fix_pressure': 0.0, 'max_tc': 100.0},
    'YBCO':  {'fix_pressure': 0.0, 'max_tc': 100.0},
    'YBa2Cu3O7': {'fix_pressure': 0.0, 'max_tc': 100.0},
    'Bi2Sr2CaCu2O8': {'fix_pressure': 0.0, 'max_tc': 110.0},
}

* 3. Từ điển ánh xạ tên gọi thông thường sang công thức hóa học

In [23]:
COMMON_NAMES_MAP = {
    'niobium': 'Nb', 'tin': 'Sn', 'hydrogen': 'H', 'sulfur': 'S', 
    'hydride': 'H', 'sulfide': 'S', 'oxide': 'O', 'boride': 'B', 
    'niobium-tin': 'Nb3Sn', 'magnesium diboride': 'MgB2', 
    'hydrogen sulfide': 'H3S', 'lanthanum hydride': 'LaH10', 
    'ybco': 'YBa2Cu3O7', 'bscco': 'Bi2Sr2CaCu2O8'
}
# Regex nhận diện công thức hóa học
FORMULA_PATTERN = re.compile(r'\b[A-Z][a-z]?\d*(?:\.[0-9]+)?(?:[A-Z][a-z]?\d*(?:\.[0-9]+)?)+\b')

# IQR FILTERING PARAMETERS

In [24]:
def remove_outliers_iqr(df):
    """
    Lọc nhiễu thống kê theo từng công thức hóa học bằng phương pháp IQR.
    Giúp loại bỏ các giá trị Tc bất thường trong cùng một nhóm vật liệu.
    """
    clean_indices = []
    for formula in df['Final_Formula'].unique():
        subset = df[df['Final_Formula'] == formula]
        if len(subset) < 5:
            clean_indices.extend(subset.index.tolist())
            continue
        Q1 = subset['Tc_Clean'].quantile(0.25)
        Q3 = subset['Tc_Clean'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 2.0 * IQR
        upper_bound = Q3 + 2.0 * IQR
        valid_subset = subset[(subset['Tc_Clean'] >= lower_bound) & (subset['Tc_Clean'] <= upper_bound)]
        clean_indices.extend(valid_subset.index.tolist())
    return df.loc[clean_indices]

def calculate_magpie_features(formula):
    zero_series = pd.Series([0]*8, index=['avg_mass','rng_mass','avg_rad','rng_rad','avg_eneg','rng_eneg','avg_val','rng_val'])
    if not isinstance(formula, str): return zero_series
    formula = re.sub(r"\s+", "", formula)
    tokens = re.findall(r'([A-Z][a-z]?)(\d*\.?\d*)', formula)
    if not tokens: return zero_series
    props = {'mass': [], 'rad': [], 'eneg': [], 'val': []}
    weights = []
    for el, num_str in tokens:
        if el not in ATOM_DATA: return zero_series
        try: num = float(num_str) if num_str and num_str != '.' else 1.0
        except: return zero_series
        weights.append(num)
        for k in props: props[k].append(ATOM_DATA[el][k])
    if not weights or sum(weights) == 0: return zero_series
    features = []
    for k in ['mass', 'rad', 'eneg', 'val']:
        vals = np.array(props[k])
        avg = np.average(vals, weights=weights)
        rng = np.max(vals) - np.min(vals)
        features.extend([avg, rng])
    return pd.Series(features, index=['avg_mass','rng_mass','avg_rad','rng_rad','avg_eneg','rng_eneg','avg_val','rng_val'])

# CÁC HÀM XỬ LÝ (HELPER FUNCTIONS)

In [25]:
def clean_tc_value(val):
    """
    Trích xuất và làm sạch giá trị nhiệt độ tới hạn (Tc).
    Xử lý trường hợp dải giá trị (ví dụ '20-30') bằng cách lấy trung bình.
    """
    if pd.isna(val): return None
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", str(val))
    if not nums: return None
    nums_float = [float(n) for n in nums]
    return sum(nums_float) / len(nums_float)

def clean_pressure_value(val):
    """
    Trích xuất và chuẩn hóa giá trị áp suất về đơn vị GPa.
    """
    if pd.isna(val): return 0.0
    val_str = str(val).lower()
    if 'ambient' in val_str: return 0.0
    
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", val_str)
    if not nums: return 0.0
    number = float(nums[0])
    
    # Chuyển đổi đơn vị
    if 'gpa' in val_str: return number
    elif 'kbar' in val_str: return number / 10.0
    elif 'mpa' in val_str: return number / 1000.0
    
    # Nếu không có đơn vị nhưng có từ khóa áp suất
    if any(u in val_str for u in ['gpa', 'kbar', 'mpa', 'pa']):
        return number
    return 0.0 

def extract_formula(row):
    """
    Trích xuất công thức hóa học từ các cột: formula -> name -> rawMaterial.
    """
    # Ưu tiên 1: Cột formula
    if pd.notna(row['formula']): return row['formula']
    # Ưu tiên 2: Cột name
    if pd.notna(row['name']): return row['name']
    
    # Ưu tiên 3: Trích xuất từ rawMaterial
    if pd.notna(row['rawMaterial']):
        raw = str(row['rawMaterial'])
        matches = FORMULA_PATTERN.findall(raw)
        # Lọc các kết quả nhiễu (quá ngắn hoặc không có số)
        valid = [m for m in matches if any(c.isdigit() for c in m) or len(m) > 2]
        if valid: return max(valid, key=len)
        
        # Ưu tiên 4: Dựa vào tên gọi thông thường
        for k, v in COMMON_NAMES_MAP.items():
            if k in raw.lower(): return v
    return None

def normalize_formula_string(f):
    """
    Chuẩn hóa chuỗi công thức: xóa khoảng trắng, ký tự lạ.
    Ví dụ: 'LaH 10' -> 'LaH10'
    """
    if pd.isna(f): return ""
    return str(f).replace(" ", "").replace("_", "").strip()

def apply_expert_logic(row):
    """
    Áp dụng logic chuyên gia để sửa áp suất và loại bỏ dữ liệu sai lệch.
    Trả về: Giá trị áp suất (float) hoặc None (nếu dòng dữ liệu cần bị loại bỏ).
    """
    f_norm = normalize_formula_string(row['Final_Formula'])
    p = row['Pressure_GPa']
    tc = row['Tc_Clean']
    
    for key, rule in EXPERT_RULES.items():
        if key.lower() in f_norm.lower():
            # 1. Sửa áp suất nếu phát hiện sai lệch (áp suất thấp cho vật liệu cao áp)
            if 'fix_pressure' in rule:
                if p < 5.0 and rule['fix_pressure'] > 50.0:
                    p = rule['fix_pressure']
            
            # 2. Kiểm tra ngưỡng Tc tối đa (loại bỏ nhiễu)
            if 'max_tc' in rule:
                if tc > rule['max_tc'] and p < 5.0:
                    return None # Đánh dấu là dữ liệu rác
            break
            
    return p

def calculate_magpie_features(formula):
    """
    Tính toán 8 đặc trưng Magpie (Mean, Range của Mass, Radius, Electronegativity, Valence).
    """
    zero_series = pd.Series([0]*8, index=['avg_mass','rng_mass','avg_rad','rng_rad','avg_eneg','rng_eneg','avg_val','rng_val'])
    if not isinstance(formula, str): return zero_series
    
    # Chuẩn hóa chuỗi
    formula = re.sub(r"\s+", "", formula)
    tokens = re.findall(r'([A-Z][a-z]?)(\d*\.?\d*)', formula)
    if not tokens: return zero_series
    
    props = {'mass': [], 'rad': [], 'eneg': [], 'val': []}
    weights = []
    
    for el, num_str in tokens:
        if el not in ATOM_DATA: return zero_series
        try: 
            num = float(num_str) if num_str and num_str != '.' else 1.0
        except: 
            return zero_series
        
        weights.append(num)
        for k in props: 
            props[k].append(ATOM_DATA[el][k])
            
    if not weights or sum(weights) == 0: return zero_series
    
    features = []
    for k in ['mass', 'rad', 'eneg', 'val']:
        vals = np.array(props[k])
        avg = np.average(vals, weights=weights)
        rng = np.max(vals) - np.min(vals)
        features.extend([avg, rng])
        
    return pd.Series(features, index=['avg_mass','rng_mass','avg_rad','rng_rad','avg_eneg','rng_eneg','avg_val','rng_val'])

# QUY TRÌNH CHÍNH (MAIN PIPELINE)

In [26]:
def main():
    input_file = "../data/data_NIMS/supercon2_v22.12.03.csv"
    output_file = "../data/data_final/SuperCon_Expert_Fix_final.csv"
    
    print("[INFO] Bắt đầu quy trình xử lý dữ liệu...")
    try:
        df = pd.read_csv(input_file)
        print(f"[INFO] Đã tải file '{input_file}'. Tổng số dòng: {len(df)}")
    except FileNotFoundError:
        print(f"[ERROR] Không tìm thấy file đầu vào '{input_file}'.")
        return

    # 1. Clean basic
    df['Tc_Clean'] = df['criticalTemperature'].apply(clean_tc_value)
    df['Pressure_GPa'] = df['appliedPressure'].apply(clean_pressure_value)
    df = df[(df['Tc_Clean'] >= 0) & (df['Tc_Clean'] <= 300)]
    
    # 2. Extract Formula
    df['Final_Formula'] = df.apply(extract_formula, axis=1)
    df = df.dropna(subset=['Final_Formula'])

    # [THÊM ĐOẠN NÀY] Chuẩn hóa công thức ngay lập tức
    print("[INFO] Đang chuẩn hóa tên công thức (xóa dấu cách thừa)...")
    df['Final_Formula'] = df['Final_Formula'].apply(normalize_formula_string)
    
    # 3. IQR Filtering 
    print(f"[INFO] Số dòng trước khi lọc IQR: {len(df)}")
    df = remove_outliers_iqr(df)
    print(f"[INFO] Số dòng sau khi lọc IQR: {len(df)} (Đã khớp với mong đợi chưa?)")

    # 4. Expert Rules
    df['Pressure_GPa_Fixed'] = df.apply(apply_expert_logic, axis=1)
    df = df.dropna(subset=['Pressure_GPa_Fixed'])
    df['Pressure_GPa'] = df['Pressure_GPa_Fixed']
    df = df.drop(columns=['Pressure_GPa_Fixed'])
    
    # 5. Physics Barrier
    print("[INFO] Đang áp dụng bộ lọc vật lý (Tc > 145K tại < 5 GPa)...")
    df = df[~((df['Pressure_GPa'] < 5.0) & (df['Tc_Clean'] > 145.0))]

    # 6. Magpie Features
    magpie_features = df['Final_Formula'].apply(calculate_magpie_features)
    cols_to_keep = ['Final_Formula', 'Tc_Clean', 'Pressure_GPa', 'id', 'rawMaterial']
    df_final = pd.concat([df[cols_to_keep], magpie_features], axis=1)
    df_final = df_final[df_final['avg_mass'] > 0]

    # Save
    df_final.to_csv(output_file, index=False)
    print(f"[SUCCESS] Hoàn tất! File '{output_file}' đã sẵn sàng. Tổng số mẫu: {len(df_final)}")

if __name__ == "__main__":
    main()

[INFO] Bắt đầu quy trình xử lý dữ liệu...
[INFO] Đã tải file '../data/data_NIMS/supercon2_v22.12.03.csv'. Tổng số dòng: 40324
[INFO] Đang chuẩn hóa tên công thức (xóa dấu cách thừa)...
[INFO] Số dòng trước khi lọc IQR: 33097
[INFO] Số dòng sau khi lọc IQR: 30192 (Đã khớp với mong đợi chưa?)
[INFO] Đang áp dụng bộ lọc vật lý (Tc > 145K tại < 5 GPa)...
[SUCCESS] Hoàn tất! File '../data/data_final/SuperCon_Expert_Fix_final.csv' đã sẵn sàng. Tổng số mẫu: 24769
